# Chatterbox TTS - Colab Notebook
This notebook demonstrates how to use Chatterbox, a state-of-the-art open-source TTS model. Follow the steps below to get started.

## 1. Install Dependencies
First, let's install the required packages.

In [ ]:
!pip install chatterbox-tts gradio

## 2. Import Libraries and Setup

In [ ]:
import torchaudio as ta
import torch
from chatterbox.tts import ChatterboxTTS

# Automatically detect the best available device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using device: {device}")

## 3. Load the Model
Now we'll load the pre-trained Chatterbox TTS model.

In [ ]:
model = ChatterboxTTS.from_pretrained(device=device)

## 4. Generate Speech from Text
Let's generate some speech using the default voice.

In [ ]:
text = "Hello! This is a test of the Chatterbox TTS system. It can generate natural-sounding speech from text."
wav = model.generate(text)

# Save the generated audio
output_path = "test_output.wav"
ta.save(output_path, wav, model.sr)

# Display audio player in the notebook
from IPython.display import Audio
Audio(output_path)

## 5. Voice Cloning (Optional)
You can also clone a voice by providing an audio prompt. Upload your audio file and specify its path below.

In [ ]:
# Upload your audio file and set its path here
AUDIO_PROMPT_PATH = "path_to_your_audio.wav"  # Replace with your audio file path

# Generate speech with the voice from the audio prompt
text = "This is the same text but spoken in a different voice."
wav = model.generate(text, audio_prompt_path=AUDIO_PROMPT_PATH)

# Save and play the generated audio
output_path_cloned = "test_output_cloned.wav"
ta.save(output_path_cloned, wav, model.sr)
Audio(output_path_cloned)

## 6. Gradio UI with Live Server

In [ ]:
import gradio as gr

def generate_with_gradio(text, audio_prompt, exaggeration, temperature, cfgw, min_p, top_p, repetition_penalty, max_new_tokens):
    try:
        # The model is already loaded in the notebook's global state
        audio_prompt_path = audio_prompt if audio_prompt else None
        wav = model.generate(
            text,
            audio_prompt_path=audio_prompt_path,
            exaggeration=exaggeration,
            temperature=temperature,
            cfg_weight=cfgw,
            min_p=min_p,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            max_new_tokens=int(max_new_tokens),
        )
        return (model.sr, wav.squeeze(0).numpy()), None
    except Exception as e:
        return None, str(e)

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            text = gr.Textbox(
                value="Now let's make my mum's favourite. So three mars bars into the pan. Then we add the tuna and just stir for a bit, just let the chocolate and fish infuse. A sprinkle of olive oil and some tomato ketchup. Now smell that. Oh boy this is going to be incredible.",
                label="Text to synthesize",
                max_lines=5
            )
            ref_wav = gr.Audio(sources=["upload", "microphone"], type="filepath", label="Reference Audio File", value=None)
            exaggeration = gr.Slider(0.25, 2, step=.05, label="Exaggeration (Neutral = 0.5, extreme values can be unstable)", value=.5)
            cfg_weight = gr.Slider(0.0, 1, step=.05, label="CFG/Pace", value=0.5)

            with gr.Accordion("More options", open=False):
                temp = gr.Slider(0.05, 5, step=.05, label="temperature", value=.8)
                min_p = gr.Slider(0.00, 1.00, step=0.01, label="min_p || Newer Sampler. Recommend 0.02 > 0.1. Handles Higher Temperatures better. 0.00 Disables", value=0.05)
                top_p = gr.Slider(0.00, 1.00, step=0.01, label="top_p || Original Sampler. 1.0 Disables(recommended). Original 0.8", value=1.00)
                repetition_penalty = gr.Slider(1.00, 2.00, step=0.1, label="repetition_penalty", value=1.2)
                max_new_tokens = gr.Slider(100, 2000, step=50, label="Max New Tokens", value=1000)

            run_btn = gr.Button("Generate", variant="primary")

        with gr.Column():
            audio_output = gr.Audio(label="Output Audio")
            error_output = gr.Textbox(label="Error", visible=False)

    def show_error(error_message):
        if error_message:
            return gr.update(visible=True, value=error_message)
        return gr.update(visible=False)

    run_btn.click(
        fn=generate_with_gradio,
        inputs=[
            text,
            ref_wav,
            exaggeration,
            temp,
            cfg_weight,
            min_p,
            top_p,
            repetition_penalty,
            max_new_tokens,
        ],
        outputs=[audio_output, error_output],
    ).then(show_error, inputs=error_output, outputs=error_output)

demo.launch(share=True)